In [ ]:
import os
import json
import subprocess
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' ### run on CPU


from cooltools.lib.numutils import set_diag
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf
import matplotlib.colors as mcolors


from basenji import dataset, dna_io, seqnn

## Load trained model

In [ ]:
### load params, specify model ###

model_dir = '/home1/yxiao977/sc1/train_akita/data/3m/train_out/'
params_file = model_dir+'params.json'
model_file  = model_dir+'model_check.h5'
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params['model']
    params_train = params['train']

seqnn_model = seqnn.SeqNN(params_model)

In [ ]:
### names of targets ###
data_dir =   '/home1/yxiao977/sc1/train_akita/data/3m/'

hic_targets = pd.read_csv(data_dir+'/targets.txt',sep='\t')
hic_file_dict_num = dict(zip(hic_targets['index'].values, hic_targets['file'].values) )
hic_file_dict     = dict(zip(hic_targets['identifier'].values, hic_targets['file'].values) )
hic_num_to_name_dict = dict(zip(hic_targets['index'].values, hic_targets['identifier'].values) )

# read data parameters
data_stats_file = '%s/statistics.json' % data_dir
with open(data_stats_file) as data_stats_open:
    data_stats = json.load(data_stats_open)
seq_length = data_stats['seq_length']
target_length = data_stats['target_length']
hic_diags =  data_stats['diagonal_offset']
target_crop = data_stats['crop_bp'] // data_stats['pool_width']
target_length1 = data_stats['seq_length'] // data_stats['pool_width']

## Make predictions for saved tfrecords

In [ ]:
### load data ###

sequences = pd.read_csv(data_dir+'sequences.bed', sep='\t', names=['chr','start','stop','type'])
sequences_test = sequences.iloc[sequences['type'].values=='test']
sequences_test.reset_index(inplace=True, drop=True)

test_data = dataset.SeqDataset(data_dir, 'test', batch_size=8)
test_inputs, test_targets = test_data.numpy()

In [ ]:

sequences_train = sequences.iloc[sequences['type'].values=='train']
sequences_train.reset_index(inplace=True, drop=True)
train_data = dataset.SeqDataset(data_dir, 'train', batch_size=8)
train_inputs, train_targets = train_data.numpy()


In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]

# Get a sorted list of the objects and their sizes
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x))/(1024**2)) ## in MegaBytes
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}

In [ ]:
### for converting from flattened upper-triangluar vector to symmetric matrix  ###

def from_upper_triu(vector_repr, matrix_len, num_diags):
    z = np.zeros((matrix_len,matrix_len))
    triu_tup = np.triu_indices(matrix_len,num_diags)
    z[triu_tup] = vector_repr
    for i in range(-num_diags+1,num_diags):
        set_diag(z, np.nan, i)
    return z + z.T

target_length1_cropped = target_length1 - 2*target_crop
print('flattened representation length:', target_length) 
print('symmetrix matrix size:', '('+str(target_length1_cropped)+','+str(target_length1_cropped)+')')

In [ ]:
train_indices = sequences_train[sequences_train['chr'] == 'chr2_pilon'].index


fig2_examples = ['chr15_pilon:10640000-10890000']
fig2_inds = []


for seq in fig2_examples:
    print(seq)
    chrm,start,stop = seq.split(':')[0], seq.split(':')[1].split('-')[0], seq.split(':')[1].split('-')[1]
    test_ind = np.where( (sequences_test['chr'].values== chrm) *
                         (sequences_test['start'].values== int(start))*
                         (sequences_test['stop'].values==  int(stop ))  )[0][0]
    fig2_inds.append(test_ind)
fig2_inds

In [ ]:
### make predictions and plot the three examples above ###
fig2_inds = [194]

target_index = 0 # HFF 

for test_index in fig2_inds:

    chrm, seq_start, seq_end = sequences_test.iloc[test_index][0:3]
    myseq_str = chrm+':'+str(seq_start)+'-'+str(seq_end)
    print(' ')
    print(myseq_str)


    test_target = test_targets[test_index:test_index+1,:,:]
    seq_length = 250000
    test_input = tf.reshape(test_inputs[test_index:test_index+1,:,:], seq_length)
    test_input = tf.one_hot(test_input, 1+4, dtype=tf.uint8)
    test_input = test_input[tf.newaxis,:,:-1] 

    test_pred = seqnn_model.model.predict(test_input)


    plt.figure(figsize=(8,4))
    target_index = 0
    vmin=-0.0010712264105677605; vmax=0.0014932174235582352

    target_length1_cropped = 50
    hic_diag = 2

    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=0)

    # plot pred
    plt.subplot(121) 
    mat = from_upper_triu(test_pred[:,:,target_index], target_length1_cropped, hic_diags)
    mat = np.nan_to_num(mat)
    im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', norm=norm)
    plt.colorbar(im, fraction=.04, pad = 0.05)
    plt.title('pred-'+str(hic_num_to_name_dict[target_index]),y=1.15 )
    plt.ylabel(myseq_str)

    # plot target 
    plt.subplot(122) 
    mat = from_upper_triu(test_target[:,:,target_index], target_length1_cropped, hic_diags)
    im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', norm=norm)
    plt.colorbar(im, fraction=.04, pad = 0.05)
    plt.title( 'target-'+str(hic_num_to_name_dict[target_index]),y=1.15)

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize test slices
test_indices = sequences_test[sequences_test['chr'] == 'chr2_pilon'].index

target_index = 0 # HFF 

vmin=-0.0010712264105677605; vmax=0.0014932174235582352
norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=0)

for test_index in test_indices[:2]:

    chrm, seq_start, seq_end = sequences_test.iloc[test_index][0:3]
    myseq_str = chrm+':'+str(seq_start)+'-'+str(seq_end)
    print(' ')
    print(myseq_str)

    test_target = test_targets[test_index:test_index+1,:,:]

    target_index = 0
    vmin=-0.0010712264105677605; vmax=0.0014932174235582352

    target_length1_cropped = 50
    hic_diag = 2

    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=0)

    # plot target 
    mat = from_upper_triu(test_target[:,:,target_index], target_length1_cropped, hic_diags)
    im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', norm=norm)
    plt.colorbar(im, fraction=.04, pad = 0.05)
    plt.title( 'target-'+str(hic_num_to_name_dict[target_index]),y=1.15)
    plt.show()

In [ ]:
# Visualize train slices 
train_indices = sequences_train[sequences_train['chr'] == 'chr1_pilon'].index

target_index = 0 # HFF 

for train_index in range(10):

    chrm, seq_start, seq_end = sequences_train.iloc[train_index][0:3]
    myseq_str = chrm+':'+str(seq_start)+'-'+str(seq_end)
    print(' ')
    print(myseq_str)

    train_target = train_targets[train_index:train_index+1,:,:]

    target_index = 0
    vmin=-0.0010712264105677605; vmax=0.0014932174235582352

    target_length1_cropped = 50
    hic_diag = 2

    norm = mcolors.TwoSlopeNorm(vmin=vmin, vmax=vmax, vcenter=0)

    # plot target 
    mat = from_upper_triu(train_target[:,:,target_index], target_length1_cropped, hic_diags)
    im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', norm=norm)
    plt.colorbar(im, fraction=.04, pad = 0.05)
    plt.title( 'target-'+str(hic_num_to_name_dict[target_index]),y=1.15)
    plt.show()

## Make a prediction from sequence

In [ ]:
### make a prediction from sequence ###

if not os.path.isfile('./data/hg38.ml.fa'):
    print('downloading hg38.ml.fa')
    subprocess.call('curl -o ./data/hg38.ml.fa.gz https://storage.googleapis.com/basenji_barnyard2/hg38.ml.fa.gz', shell=True)
    subprocess.call('gunzip ./data/hg38.ml.fa.gz', shell=True)

fasta_open = pysam.Fastafile('./data/hg38.ml.fa')

In [ ]:
# this example uses the sequence for the test set region
# with the corresponding test_index, but
# predictions can be made for any DNA sequence of length = seq_length = 2^20

chrm, seq_start, seq_end = sequences_test.iloc[test_index][0:3] 
seq = fasta_open.fetch( chrm, seq_start, seq_end ).upper()
if len(seq) != seq_length: raise ValueError('len(seq) != seq_length')

# seq_1hot is a np.array with shape [2^20 bp, 4 nucleotides]
# representing 1-hot encoded DNA sequence
seq_1hot = dna_io.dna_1hot(seq)

In [ ]:
# expand input dimensions, as model accepts arrays of size [#regions,2^20bp, 4]
test_pred_from_seq = seqnn_model.model.predict(np.expand_dims(seq_1hot,0))

In [ ]:
# plot pred

plt.figure(figsize=(8,4))
target_index = 0
vmin=-2; vmax=2

#transform from flattened representation to symmetric matrix representation
mat = from_upper_triu(test_pred_from_seq[:,:,target_index], target_length1_cropped, hic_diags)

plt.subplot(121) 
im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
plt.colorbar(im, fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
plt.title('pred-'+str(hic_num_to_name_dict[target_index]),y=1.15 )
plt.ylabel(myseq_str)

# plot target 
plt.subplot(122) 
mat = from_upper_triu(test_target[:,:,target_index], target_length1_cropped, hic_diags)
im = plt.matshow(mat, fignum=False, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
plt.colorbar(im, fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
plt.title( 'target-'+str(hic_num_to_name_dict[target_index]),y=1.15)

plt.tight_layout()
plt.show()